# Capstone Final Project

#### Imports

In [283]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

#! pip install folium==0.5.0
import folium # plotting library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#### Get Latitude and Longitude

In [284]:
# Find Lat/Long for 10 Colorado cities
geolocator = Nominatim(user_agent="co_explorer")
co_cities = ['Colorado Springs', 'Fort Collins', 'Lakewood', 'Thornton', 'Arvada', 'Westminster', 'Centennial']
co_locations = []
for city in co_cities:
    address = city + ', CO'
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    co_locations.append({'City':city, 'Latitude':latitude, 'Longitude':longitude})

#Denver, Aurora and Pueblo are WRONG in Nominatim, so manually add these
denver_lat = 39.7493892726503
denver_long = -105.01281319544394
co_locations.append({'City':'Denver', 'Latitude':denver_lat, 'Longitude':denver_long})
co_locations.append({'City':'Aurora', 'Latitude':39.71886255122142, 'Longitude':-104.81181164668811})
co_locations.append({'City':'Pueblo', 'Latitude':38.255093674315326, 'Longitude':-104.60666349650079})

#### Map of 10 Colorado Cities

In [285]:
venues_map = folium.Map(location=[denver_lat, denver_long], zoom_start=8) # generate map 

# put a marker on each city
for location in co_locations:
    folium.CircleMarker([location.get('Latitude'), location.get('Longitude')],radius=10, color='purple', popup=location.get('City'),
                       fill=True, fill_color='purple', fill_opacity=0.6).add_to(venues_map)

# display map
venues_map

<h4><center>Graph of Denver</center></h4>

#### Hide FourSquare Credentials

In [286]:
# The code was removed by Watson Studio for sharing.

#### Get Medical Centers for Each City

In [287]:
VERSION = '20180604'
LIMIT = 50 # seems to be max value for free accounts
CATEGORY_ID = '4bf58dd8d48988d104941735' # "Medical Center", possibly including sub-categories
RADIUS = 8000; # about 5 mile radius

med_centers = {}
for location in co_locations:
    latitude = location.get("Latitude")
    longitude = location.get("Longitude")
    url='https://api.foursquare.com/v2/venues/search?categoryId={}&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&limit={}&radius={}'.format(
        CATEGORY_ID, CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, LIMIT, RADIUS)

    results = requests.get(url).json()
    venues_list = results["response"]["venues"]
    med_centers[location.get("City")] = venues_list

#### Explore Venues for a Single City

In [289]:
venues = med_centers.get('Denver')

rows = []
for venue in venues:
    categories = venue['categories']
    for category in categories:
        rows.append({'City':'Denver', #just examining single city
                    'Latitude':venue['location']['lat'],
                    'Longitude':venue['location']['lng'],
                    'VenueName':venue['name'],
                    'VenueId':venue['id'],
                    'Category':category['name']})

# turn it into dataframe
df = pd.DataFrame(rows)
df.head()

,City,Latitude,Longitude,VenueName,VenueId,Category
0,Denver,39.745126,-105.011251,Truecare24 Colorado Vaccine Site,60706e25cf76e03d4953d571,Medical Center
1,Denver,39.717922,-104.942823,Colorado Denistry - Cherry Creek,6065e2398c1c5c60e307326e,Dentist's Office
2,Denver,39.759492,-105.025750,Nuture: A Wellcare Marketplace,5f05fa8124d225059b52c83c,Gym / Fitness Center
3,Denver,39.692863,-105.025610,Southwest Medical Center,5755a6e7498e603bd01226f7,Urgent Care Center
4,Denver,39.769683,-105.029761,AFC Urgent Care,58acaa700b565621cc3c3cd0,Urgent Care Center


#### Visualize Single City Facilities

In [290]:
denver_map = folium.Map(location=[denver_lat, denver_long], zoom_start=12) # generate map centred around Ecco

# add medical facilities as blue circle markers
for index, facility in df.head(10).iterrows():
    latitude = facility['Latitude']
    longitude = facility['Longitude']
    name = facility['VenueName']
    category = facility['Category']
    folium.CircleMarker([latitude, longitude], radius=10, popup=name, fill=True, color='blue', fill_color='blue', fill_opacity=0.6).add_to(denver_map)
                                                                           
# display map
denver_map

<h4><center>Graph of 10 Medical Facilities Near Denver</center></h4>

#### Get only the data we need, one row per venue for now

In [291]:
# Create dataframe with all venues combined, one row per venue
all_med_centers = []
for city, vlist in med_centers.items():
    for ven in vlist:
        ven_name = ven.get('name')
        ven_lat = ven.get('location').get('lat')
        ven_long = ven.get('location').get('lng')
        ven_cat_list = ven.get('categories')
        for cat in ven_cat_list:
            cat_name = cat.get('name')
            all_med_centers.append({'City':city, 'VenueName':ven_name, 'Latitude': ven_lat, 'Longitude': ven_long, 'CategoryName':cat_name})
            
ven_df = pd.DataFrame(all_med_centers)
print(ven_df.shape)
ven_df.tail()


(486, 5)


,City,VenueName,Latitude,Longitude,CategoryName
481,Pueblo,Belaire Animal Hospital,38.222562,-104.643376,Veterinarian
482,Pueblo,Fresenius Medical Care,38.215035,-104.625187,Medical Center
483,Pueblo,LabCorp,38.309959,-104.659215,Medical Lab
484,Pueblo,St. Mary Corwin Hospital,38.233611,-104.624190,Hospital
485,Pueblo,Parkview Medical Center,38.282273,-104.612417,Hospital


#### See all of the category names

In [292]:
ven_df['CategoryName'].unique()

array(["Doctor's Office", 'Hospital', "Dentist's Office",
       'Medical Center', 'Urgent Care Center', 'Mental Health Office',
       'Assisted Living', 'Chiropractor', 'Veterinarian', 'Spa',
       'Emergency Room', 'Pet Service', 'Marijuana Dispensary', 'Gym',
       'Drugstore', 'Medical Lab', 'Herbs & Spices Store',
       'Alternative Healer', 'Office', 'Gym / Fitness Center',
       'Eye Doctor', 'Physical Therapist', 'Maternity Clinic',
       'Optical Shop', 'Acupuncturist', 'Community Center', 'Pharmacy',
       'Vape Store', 'Pet Store', 'Medical School', 'Rehab Center',
       'Daycare'], dtype=object)

#### Get counts of category names per city

In [293]:
# Break down into category counts per city
cat_counts = ven_df.groupby(['City', 'CategoryName'], as_index=False).count()
cat_counts.rename(columns={"VenueName":"Count"}, inplace=True)
del cat_counts['Latitude']
del cat_counts['Longitude']

# Finally have what we want -- just the City, Category Name, and Count of those types of Medical Facilities
cat_counts.tail(10)

,City,CategoryName,Count
128,Westminster,Hospital,1
129,Westminster,Marijuana Dispensary,3
130,Westminster,Medical Center,3
131,Westminster,Medical Lab,1
132,Westminster,Mental Health Office,1
133,Westminster,Optical Shop,2
134,Westminster,Physical Therapist,3
135,Westminster,Spa,1
136,Westminster,Urgent Care Center,2
137,Westminster,Veterinarian,3


In [294]:
# Ignore categories for a city if fewer than 5 of that facility type
top_cats = cat_counts[cat_counts['Count'] >= 5]
top_cats.head(20)

,City,CategoryName,Count
4,Arvada,Dentist's Office,9
5,Arvada,Doctor's Office,15
9,Arvada,Medical Center,5
16,Aurora,Dentist's Office,6
17,Aurora,Doctor's Office,8
20,Aurora,Hospital,12
22,Aurora,Medical Center,7
31,Centennial,Chiropractor,5
32,Centennial,Dentist's Office,7
33,Centennial,Doctor's Office,12


In [295]:
# which categories are left if we ignore those with fewer than 5 facilities per area
top_names = top_cats['CategoryName'].unique()
all_cities = top_cats['City'].unique()
print(top_names)

["Dentist's Office" "Doctor's Office" 'Medical Center' 'Hospital'
 'Chiropractor' 'Veterinarian']


In [297]:
# which cities have more than 5 veternarians
vets_df = top_cats[top_cats['CategoryName'] == "Veterinarian"]
vets_df.head()

,City,CategoryName,Count
81,Fort Collins,Veterinarian,8
94,Lakewood,Veterinarian,8


In [298]:
# Create DataFrame in format for graphs counts by type per city
per_cat_data = {}
for category in top_names:
    counts_one_cat = []
    for city in all_cities:
        cat_city_count_series = top_cats.loc[(top_cats['CategoryName'] == category) & (top_cats['City'] == city)]['Count']
        if len(cat_city_count_series) > 0:
            counts_one_cat.append(cat_city_count_series.values[0])
        else:
            counts_one_cat.append(0)
    per_cat_data[category] = counts_one_cat 

per_cat_df = pd.DataFrame(per_cat_data, columns = top_names)
per_cat_df['City'] = all_cities 
per_cat_df.head(10)


,Dentist's Office,Doctor's Office,Medical Center,Hospital,Chiropractor,Veterinarian,City
0,9,15,5,0,0,0,Arvada
1,6,8,7,12,0,0,Aurora
2,7,12,6,0,5,0,Centennial
3,0,16,15,6,0,0,Colorado Springs
4,0,15,10,5,0,0,Denver
5,12,14,8,0,0,8,Fort Collins
6,11,9,0,0,0,8,Lakewood
7,10,11,6,0,0,0,Pueblo
8,6,18,6,0,0,0,Thornton
9,7,20,0,0,0,0,Westminster


#### Graph Counts of Facility Types per City

In [299]:
all_bars = []
for city in all_cities:
    city_info = per_cat_df.loc[per_cat_df['City'] == city]
    city_series = city_info.values[0].tolist()[0:6]

    city_bar = go.Bar(name=city, x=top_names, y=city_series)
    all_bars.append(city_bar)

fig = go.Figure(data=all_bars)
fig.update_layout(barmode='group')
fig.update_layout(title_text='Numbers of Facility Types by City')

fig.show()

#### Graph of City Facility Types

In [300]:
fig2 = go.Figure()

for category in top_names:
    X = per_cat_df[category]
    cat_bar = go.Bar(y=all_cities, x=X, name=category, orientation='h') #not sure if need marker part

    fig2.add_trace(cat_bar)

fig2.update_layout(barmode='stack')
fig2.update_layout(title_text='Types of Facilities in Each City')
fig2.show()    